## Final Project - CIFAR10

1.   Using EfficentNetB0 to extract rich features
2.   Using dense neural network to clasify our classes


In [ ]:
import keras
import matplotlib
from keras import datasets
from tensorflow.keras.applications import EfficientNetB0
import numpy as np
import random
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
import cv2


1.   Load CIFAR10 Dataset using Keras library 
2.   Apply the augmentation on training data
3.   Use to_categorical method to change data labels to one-hot form

Note: For EfficientNetB0 input data should range [0, 255]. Normalization is included as part of the model, so our data without preprossening is already range [0, 255]

In [ ]:
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

train_datagen = ImageDataGenerator(
        rotation_range=5,
        width_shift_range=0.2,
        height_shift_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

train_datagen.fit(x_train)

test_datagen = ImageDataGenerator()
test_datagen.fit(x_test)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Download and set EfficientNetB0 as pretrain networks

1.   We set include_top = false for both models becuase we want to those for feature extacting
2.   We consider Imagenet as an initial weight

Note: We know EfficientNetB0 were trained on ImageNet dataset that each data has 224 * 224 * 3 dimensions so we needed to increase CIFAR10's size of dimensionality. Because of memory exception in colab, we decided to increase that 4 times (128, 128, 3) by Upsamplaing in our model.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, UpSampling2D, GlobalAveragePooling2D, Flatten, Concatenate
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from keras.models import Model

efficient_model = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=None, input_shape=(128, 128, 3))

for layer in efficient_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else: 
        layer.trainable = False

In [ ]:
# Model configuration
batch_size = 32
loss_function = categorical_crossentropy
num_classes = 10

eff_model = Sequential()
eff_model.add(UpSampling2D(input_shape=(32, 32, 3)))
eff_model.add(UpSampling2D())
eff_model.add(efficient_model)
eff_model.add(GlobalAveragePooling2D())
eff_model.add(Dense(512, activation='relu'))
eff_model.add(Dropout(.5))
eff_model.add(Dense(512, activation='relu'))
eff_model.add(Dropout(.5))
eff_model.add(Dense(num_classes, activation='softmax'))

# Compile the model
eff_model.compile(loss=loss_function,
                  optimizer = Adam(),
                  metrics=['accuracy'])

eff_model.summary()

history = eff_model.fit(
              train_datagen.flow(x_train,y_train, batch_size=32),
              epochs = 10,
              verbose = 1,
              batch_size = batch_size,
              validation_data = test_datagen.flow(x_test, y_test, batch_size=32)
            )

# Generate generalization metrics
score = eff_model.evaluate(x_test, y_test, verbose=0)
print('Test loss: {0} / Test accuracy: {1}'.format(score[0], score[1]))

# Visualize history
# Plot history: Loss
plt.plot(history.history['val_loss'])
plt.title('Validation loss history')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.show()

# Plot history: Accuracy
plt.plot(history.history['val_accuracy'])
plt.title('Validation accuracy history')
plt.ylabel('Accuracy value (%)')
plt.xlabel('No. epoch')
plt.show()